In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from pyearth import Earth
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score

In [2]:
data = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')
data.columns = [x.strip() for x in data.columns.tolist()]
X = data.drop(columns = ['url', 'timedelta', 'shares'])
y = data['shares']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)

In [3]:
X_train = X_train.drop(columns = ['weekday_is_monday',
 'weekday_is_saturday',
 'LDA_00',
 'n_unique_tokens',
 'n_non_stop_words',
 'self_reference_avg_sharess',
 'rate_positive_words',
 'LDA_04',
 'LDA_02',
 'kw_max_max',
 'kw_avg_min',
 'kw_avg_avg',
 'rate_negative_words',
 'min_negative_polarity'])

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from pyearth import Earth

# Define the model
mars_model = Earth()

# Define the parameters
parameters = {'max_terms': [300,500,1000],
              'max_degree': [1,2,3,4]}

# Define the cross-validator
CV = KFold(n_splits = 5, shuffle=True, random_state=1)

# Initialize the GridSearchCV object
model = GridSearchCV(estimator=mars_model, 
                     param_grid=parameters, 
                     cv=CV, 
                     n_jobs=-1, 
                     verbose=1, 
                     scoring='neg_mean_absolute_error')

# Fit the GridSearchCV object to the data
model.fit(X_train, y_train)  

# Print the best score and best parameters
print('Best score:', model.best_score_)
print('Best parameters:', model.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [ ]:
mars_model = Earth(max_terms = 300, max_degree = 1)
mars_model = TransformedTargetRegressor(mars_model, func = np.log, inverse_func = np.exp).fit(X_train, y_train)

In [ ]:
#K-fold cross validation to find optimal parameters for CatBoost regressor
start_time = time.time()
param_grid = {'max_depth': [4,6,8],
              'num_leaves': [20, 31, 40],
              'learning_rate': [0.01, 0.05, 0.1],
               'reg_lambda':[0, 10, 100],
                'n_estimators':[100, 500, 1000],
                'subsample': [0.5, 0.75, 1.0]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=CatBoostRegressor(random_state=1, verbose=False),                                                       
                             param_distributions = param_grid, n_iter = 200,
                             verbose = 1,random_state = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X_train,y_train)
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation R-squared = ",optimal_params.best_score_)
print("Time taken = ", round((time.time()-start_time)/60), " minutes")

In [ ]:
# Instantiate the model with the provided parameters
cat_model = CatBoostRegressor(subsample = 0.75, reg_lambda=10, 
                              depth=6, 
                              n_estimators=1000, 
                              learning_rate=0.01,
                              random_state=1, 
                              verbose=False)

# Fit the model to your training data
cat_model= TransformedTargetRegressor(cat_model, func = np.log, inverse_func = np.exp).fit(X_train, y_train)

# Predict on your test set
y_pred = cat_model.predict(X_test)  

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)  

print('Mean Absolute Error:', mae)